In [1]:
import time
import numpy as np
import pandas as pd

import requests

In [4]:
df = pd.read_excel(r'..\datasets\250716.xlsx')
print(df.head())

# df = pd.read_csv(r'..\datasets\df_na.csv', encoding = 'cp949')
# print(df.head())

   순번                   출발지 주소              중간집결지 주소            최종 도착지 주소  \
0   1  경기도 고양시 덕양구 중앙로633번길 12   경기도 고양시 덕양구 혜음로 142  경기도 고양시 덕양구 혜음로 142   
1   2   대구광역시 서구 북비산로61길 20-24      경상북도 경주시 태종로 516  경기도 고양시 덕양구 동헌로 305   
2   3   서울특별시 양천구 목동중앙본로11길 19  서울특별시 서초구 사평대로45길 26  강원특별자치도 동해시 대동로 430   
3   4        서울특별시 강남구 학동로 432       경기도 과천시 중앙로 294      경기도 과천시 중앙로 294   
4   5         대구광역시 남구 명덕로 236      대구광역시 수성구 무열로 56     대구광역시 수성구 무열로 56   

    중간집결 기한    최종도착 기한 이동방법  
0  0+1일 14시   0+1일 14시   개별  
1  0+28일 9시  0+28일 14시   집단  
2   0+3일 8시   0+3일 14시   집단  
3  0+1일 14시   0+1일 14시   개별  
4  0+3일 14시   0+3일 14시   개별  


In [5]:
num = np.array(df['순번'])

start = np.array(df['출발지 주소'])
mid = np.array(df['중간집결지 주소'])
end = np.array(df['최종 도착지 주소'])

# start = np.array(df['출발주소_수정'])
# mid = np.array(df['중간주소_수정'])
# end = np.array(df['최종주소_수정'])

In [ ]:
# def geocoding(address: list):
#     url = "https://maps.apigw.ntruss.com/map-geocode/v2/geocode"
    
#     headers = {
#         "x-ncp-apigw-api-key-id": "oa77i9oz1h",
#         "x-ncp-apigw-api-key": "xvC9h8wAZLXjsokASCSKjLfNJ5uR63sKBGz705KA",
#         "Accept": "application/json"
#     }

#     params = {"query": address}
#     resp = requests.get(url, headers = headers, params = params)
#     resp.raise_for_status()
#     data = resp.json()

#     if data.get('addresses'):
#         lat = float(data['addresses'][0]['y']) # 위도
#         lng = float(data['addresses'][0]['x']) # 경도  
#         return lat, lng
    
#     else:
#         lat = None
#         lng = None
#         return lat, lng

In [ ]:
def geocoding(address: list):
    url = "https://maps.apigw.ntruss.com/map-geocode/v2/geocode"
    
    headers = {
        "x-ncp-apigw-api-key-id": "oa77i9oz1h",
        "x-ncp-apigw-api-key": "xvC9h8wAZLXjsokASCSKjLfNJ5uR63sKBGz705KA",
        "Accept": "application/json"
    }

    params = {"query": address}
    resp = requests.get(url, headers = headers, params = params)
    resp.raise_for_status()
    data = resp.json()

    try:
        # if data.get('addresses'):
        lat = float(data['addresses'][0]['y']) # 위도
        lng = float(data['addresses'][0]['x']) # 경도
        return lat, lng
    
    except Exception as e:
        print(f'{address} 변환 실패: {e}')

        lat = None
        lng = None
        return lat, lng


In [ ]:
# geocoding
if __name__ == "__main__":
    chunk = 1000
    start_index = 0
    last_index = 10
 
    for i in range(start_index, last_index, chunk):
        slc = slice(i, min(i + chunk, last_index))
        print(f'Processing records {slc.start} to {slc.stop}...')

        t0 = time.time()
        start_lat, start_lng = zip(*(geocoding(addr) for addr in start[slc]))
        mid_lat, mid_lng = zip(*(geocoding(addr) for addr in mid[slc]))
        end_lat, end_lng = zip(*(geocoding(addr) for addr in end[slc]))
        t1 = time.time()

        print(f'걸린시간(분): {(t1-t0)/60:.2f}')

        df_chunk = pd.DataFrame({'num': num[slc],
                       'start_lat': start_lat,
                       'start_lng': start_lng,
                       'mid_lat': mid_lat,
                       'mid_lng': mid_lng,
                       'end_lat': end_lat,
                       'end_lng': end_lng})
        
        mode = 'w' if i == start_index else 'a'
        header = True if i == start_index else False

        df_chunk.to_csv(r'..\results\geocoded.csv', mode = mode, header = header, index = False)
        print(f'{slc.start} to {slc.stop} 저장 완료')

Processing records 0 to 10...
걸린시간(분): 0.05
0 to 10 저장 완료
